# PixelBytes: Catching Insights in Unified Multimodal Sequences

Ce notebook présente **PixelBytes**, un modèle innovant conçu pour générer simultanément du texte et des images pixel par pixel sous forme de séquences. L'objectif est d'explorer un embedding unifié qui permet une génération multimodale cohérente.

## Contexte et Architecture Proposée

### Fondements Théoriques
- **Image Transformer** : [Génération d'images pixel par pixel](https://arxiv.org/abs/1802.05751)
- **Bi-Mamba+** : [Modèle bidirectionnel pour la prévision de séries temporelles](https://arxiv.org/abs/2404.15772)
- **MambaByte** : [Modèle d'état d'espace sélectif sans token](https://arxiv.org/abs/2401.13660)

### Concept Clé
Le modèle PixelByte génère des séquences mixtes de texte et d'images. Il doit :
- Gérer les transitions entre texte et image avec des sauts de ligne (ASCII 0A).
- Maintenir la cohérence des dimensions des images générées.
- Assimiler la tâche de "copie" pour reproduire des motifs complexes.

Ce notebook utilise la puissance des GPU T4 x2 de Kaggle pour expérimenter avec des architectures avancées et des jeux de données volumineux, afin de relever les défis de la génération multimodale unifiée.

## Ressources du Projet

### Dataset
Pour ce projet, nous utiliserons le dataset **PixelBytes-Pokemon**, spécialement conçu pour cette tâche de génération multimodale. Ce dataset, créé par l'auteur de ce notebook, est disponible sur Hugging Face : [PixelBytes-Pokemon](https://huggingface.co/datasets/ffurfaro/PixelBytes-Pokemon). Il contient des séquences de texte et d'images de Pokémon, encodées de manière à permettre l'entraînement de notre modèle PixelByte sur des données multimodales.

### Implémentation
L'implémentation du modèle et les scripts d'entraînement sont disponibles dans le dépôt GitHub **Mamba-Bys** : [Mamba-Bys](https://github.com/fabienfrfr/Mamba-Bys). Ce dépôt contient le code source nécessaire pour reproduire les expériences, ainsi que des instructions détaillées sur la configuration et l'utilisation du modèle PixelByte.

## Modele à entrainer :

- 8 LSTM (bidirectionnel + 1,2,3 layers) + (p_embed + bi-2 layers)
- 6 Mamba (bidirectionnel + 1,2,3 layers)
- 3 Transformers (1,2,3 layers)

# Pre-test

Avant d'entrainer les 8 LSTM, prendre le pembed-bi-2 LSTM et tester la génération (à 40%, mais influence des répétitions de caractére et de pixel)

Forte chance de predire le meme pixel à la suite, et de prédire des espaces et voyelles.

In [1]:
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from pixelbytes.model import *
from pixelbytes.train import *
from pixelbytes.dataset import *
from pixelbytes.tokenizer import *

from datasets import load_dataset

In [3]:
# init
hf_dataset = load_dataset("ffurfaro/PixelBytes-Pokemon")
ds = hf_dataset["train"].train_test_split(test_size=0.1)

train_dataset = PxByDataset(ds["train"]["pixelbyte"], seq_length=256, stride=32)
test_dataset = PxByDataset(ds["test"]["pixelbyte"], seq_length=256, stride=32)

pixelbyte = PixelBytesTokenizer()

Generating train split:   0%|          | 0/964 [00:00<?, ? examples/s]

In [7]:
pixelbyte = PixelBytesTokenizer()
vocab_size = len(pixelbyte.vocab); print(vocab_size)
embedding_dim = 81
hidden_dim = 64
n_layer = 2
    
config = ModelConfig(dim=embedding_dim, d_state=hidden_dim, depth=n_layer, vocab_size=vocab_size)

113


In [10]:
#model = SimpleRNNModel(config)
model = SimpleTransformerModel(config)

In [11]:
# Utilisation
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    batch_size=32,
    learning_rate=0.001,
    num_epochs=200,
    save_dir='models',
    model_name="SimpleSeqModel",
    dataset_name="PixelBytes-Pokemon",
    eval_every=1 # Évaluer tous les 5 epochs
)

trainer.train_and_evaluate()

Test Loss: 4.9152, Accuracy: 0.83%, F1: 0.0018


Training: 100%|██████████| 732/732 [00:12<00:00, 58.61it/s]


Test Loss: 2.0879, Accuracy: 40.13%, F1: 0.3564
Model saved to models/SimpleSeqModel_PixelBytes-Pokemon/best_model.pth
Epoch 1/200, Train Loss: 2.4042


Training: 100%|██████████| 732/732 [00:12<00:00, 58.61it/s]


Test Loss: 1.9454, Accuracy: 43.17%, F1: 0.4032
Model saved to models/SimpleSeqModel_PixelBytes-Pokemon/best_model.pth
Epoch 2/200, Train Loss: 1.9818


...

Test Loss: 2.6310, Accuracy: 42.82%, F1: 0.4211
Epoch 198/200, Train Loss: 0.8006


Training: 100%|██████████| 732/732 [00:13<00:00, 55.75it/s]


Test Loss: 2.6236, Accuracy: 43.01%, F1: 0.4232
Epoch 199/200, Train Loss: 0.8028


Training: 100%|██████████| 732/732 [00:13<00:00, 55.75it/s]


Test Loss: 2.6605, Accuracy: 43.05%, F1: 0.4265
Epoch 200/200, Train Loss: 0.7906
Training results saved.
